### Libs

In [18]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os
import torch
import math
# main libraries
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
from torch.utils.data import SubsetRandomSampler
from torch.nn import functional as F
from torch import nn
from torch import optim
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import Dataset
import math
import torchvision.transforms.functional as TF
import cv2
from keras.utils import image_dataset_from_directory
from torchvision import datasets, transforms
from torch.utils.data import random_split
from torch.nn.parameter import Parameter

from torch.nn import init
from torchvision.utils import make_grid

# UNet

### Encoder

In [43]:
from collections import OrderedDict

import torch
import torch.nn as nn


class Encoder(nn.Module):

    def __init__(self, in_channels, init_features):
        super(Encoder, self).__init__()

        features = init_features
        self.encoder1 = Encoder._block(in_channels, features, name="enc1")
        self.encoder2 = Encoder._block(features, features * 2, name="enc2")
        self.encoder3 = Encoder._block(features * 2, features * 4, name="enc3")
        self.encoder4 = Encoder._block(features * 4, features * 8, name="enc4")


    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(enc1)
        enc3 = self.encoder3(enc2)
        enc4 = self.encoder4(enc3)

        return enc4, enc3, enc2, enc1



    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                    (name + "pooling",nn.MaxPool2d(kernel_size=2, stride=2))
                ]
            )
        )

In [39]:
# TEST
N, C, H, W = 1, 3, 200, 200
encoder = Encoder(3)
x = torch.ones(N, C, H, W)
encoder(x).shape

TypeError: __init__() missing 1 required positional argument: 'init_features'

### Decoder

In [45]:
from collections import OrderedDict

import torch
import torch.nn as nn


class Decoder(nn.Module):

    def __init__(self, init_features, out_channels):
        super(Decoder, self).__init__()

        features = init_features

        self.upconv4 = nn.ConvTranspose2d(features * 16, features * 8, kernel_size=2, stride=2)
        self.decoder4 = Decoder._block((features * 8) * 2, features * 8, name="dec4")

        self.upconv3 = nn.ConvTranspose2d(features * 8, features * 4, kernel_size=2, stride=2)
        self.decoder3 = Decoder._block((features * 4) * 2, features * 4, name="dec3")

        self.upconv2 = nn.ConvTranspose2d(features * 4, features * 2, kernel_size=2, stride=2)
        self.decoder2 = Decoder._block((features * 2) * 2, features * 2, name="dec2")

        self.upconv1 = nn.ConvTranspose2d(features * 2, features, kernel_size=2, stride=2)
        self.decoder1 = Decoder._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(in_channels=features, out_channels=out_channels, kernel_size=1)

    def forward(self, bottleneck, enc4, enc3, enc2, enc1):
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)

        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)

        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)

        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)

        return torch.sigmoid(self.conv(dec1))

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

In [46]:
# my unet

In [41]:
from collections import OrderedDict

import torch
import torch.nn as nn


class MyUNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(MyUNet, self).__init__()

        self.encoder = Encoder(in_channels, init_features)
        self.bottleneck = MyUNet._block(init_features * 8, init_features * 16, name="bottleneck")
        self.decoder = Decoder(init_features, out_channels)


    def forward(self, x):

        enc4, enc3, enc2, enc1 = self.encoder(x)
        bottleneck = self.bottleneck(enc4)
        x_hat = self.decoder(bottleneck, enc4, enc3, enc2, enc1)

        return torch.sigmoid(self.conv(x_hat))

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

In [42]:
# Test
N, C, H, W = 10, 3, 256, 256
m = MyUNet(3, 3)
x = torch.ones(N, C, H, W)
m(x).shape

RuntimeError: Sizes of tensors must match except in dimension 1. Expected size 32 but got size 16 for tensor number 1 in the list.

In [47]:
from collections import OrderedDict

import torch
import torch.nn as nn


class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")

        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = UNet._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNet._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNet._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )

    def forward(self, x):
        enc1 = self.encoder1(x)
        enc2 = self.encoder2(self.pool1(enc1))
        enc3 = self.encoder3(self.pool2(enc2))
        enc4 = self.encoder4(self.pool3(enc3))

        bottleneck = self.bottleneck(self.pool4(enc4))

        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat((dec4, enc4), dim=1)
        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, enc3), dim=1)
        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, enc2), dim=1)
        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, enc1), dim=1)
        dec1 = self.decoder1(dec1)
        return torch.sigmoid(self.conv(dec1))

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

In [ ]:
nn.Linear

In [50]:
m = UNet()
m.__dict__

{'training': True,
 '_parameters': OrderedDict(),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),
 '_modules': OrderedDict([('encoder1',
               Sequential(
                 (enc1conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                 (enc1norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                 (enc1relu1): ReLU(inplace=True)
                 (enc1conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                 (enc1norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                 (enc1relu2): ReLU(inplace=True)
               )),
              ('pool1',
               Max

In [80]:
class MyLinear(nn.Module):
    def __init__(self, in_features: int, out_features: int, bias: bool = True,
                 device=None, dtype=None) -> None:
        factory_kwargs = {'device': device, 'dtype': dtype}
        super(MyLinear, self).__init__()
        self.in_features = Parameter(torch.tensor(in_features).to(torch.float))
        self.out_features = out_features
        #self.weight = Parameter(torch.empty((out_features, in_features), **factory_kwargs))
        self.weight = torch.empty((out_features, in_features), **factory_kwargs)
        if bias:
            #self.bias = Parameter(torch.empty(out_features, **factory_kwargs))
            self.bias = torch.empty(out_features, **factory_kwargs)
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self) -> None:
        # Setting a=sqrt(5) in kaiming_uniform is the same as initializing with
        # uniform(-1/sqrt(in_features), 1/sqrt(in_features)). For details, see
        # https://github.com/pytorch/pytorch/issues/57109
        init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in) if fan_in > 0 else 0
            init.uniform_(self.bias, -bound, bound)

    def forward(self, input):
        self.input = Parameter(input).requires_grad_(False)
        return F.linear(input, self.weight, self.bias)

In [81]:
m = MyLinear(10,20)
x = torch.ones(1, 10).to(torch.float)
m(x)

tensor([[ 0.9415, -0.9959,  0.2554, -0.2785,  0.8067,  0.1665, -0.1722,  0.3644,
          0.2689, -0.1856,  0.8837,  0.1660, -0.1872, -0.0526,  0.5000,  0.1944,
          0.3945, -0.1017, -0.7707,  0.5823]])

In [82]:
for i in m.parameters():
    print(i.requires_grad)

True
False


# End UNet

In [15]:
A = torch.empty(100, 3, 200, 200)
A[:, 1, 100, 100].shape

torch.Size([100])

### Dataloader

In [2]:
class Anomaly_Dataset(Dataset):
    def __init__(self,
                 root
                 ):
        super(Anomaly_Dataset, self).__init__()

        self.data = Anomaly_Dataset.load_dataset(root)
        self.image, self.label = Anomaly_Dataset.get_numpy(self.data)

    def __getitem__(self, item):
        x, y =  self.image[item], self.label[item]

        # RGB -> GRAY : (H, W)
        x = x[:,:,0]

        # (1, H, W)
        x = Anomaly_Dataset.normalization(x)
        #x = np.expand_dims(x, axis=0)

        return x, y


    def __len__(self):
        return len(self.data)


    @staticmethod
    def load_dataset(path):
        img_rows = 135
        img_cols = 135
        return image_dataset_from_directory(directory = path,
                                               label_mode = 'int',
                                               color_mode = 'rgb',
                                               shuffle = False,
                                               batch_size = None,
                                               image_size = (img_rows, img_cols),
                                               crop_to_aspect_ratio = True)

    @staticmethod
    def get_numpy(PrefetchDataset):
        """
        return:
            (N, H, W, C) , (N,)
        """
        images = []
        labels = []
        for (image, label) in PrefetchDataset:
            images.append(image)
            labels.append(label)
        return np.array(images), np.array(labels)

    @staticmethod
    def rgb_2_gray(x):
        """
        (H, W, C) --> (H, W)
        """
        return cv2.cvtColor(x, cv2.COLOR_BGR2GRAY)

    @staticmethod
    def normalization(x):
        """
        Args:
            x : np.array : (H, W)

        Return:
            np.array : (H, W)
        """
        x = x - x.min(keepdims=True)
        x = x / x.max(keepdims=True)
        x = x - 0.5
        return  x / 0.5

### Noise

In [3]:
def get_noise(num_z, z_dim, device='cpu'):
    # mean, std = 0, 1
    #return torch.empty(num_z, z_dim, 1, 1).normal_(mean, std).to(device)
    return torch.randn(num_z, z_dim, 1, 1, device=device)

In [4]:
def get_noise(num_z, z_dim, device='cpu'):

  steps_ = z_dim * (4*10**(-4))

  sample_s = []
  for _ in range(num_z):
    sample = torch.normal(torch.arange(-0.5, 0.5, steps_), torch.arange(0,0.5,steps_/2)).unsqueeze(0)
    sample_s += [sample]

  return torch.concat(sample_s, dim=0).unsqueeze(-1).unsqueeze(-1)

### Net

In [ ]:
# Change activation to RELU
class Generator(nn.Module):
    def __init__(self, z_dim , c_dim , gf_dim):
        super(Generator, self).__init__()

        # torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0,
        #                          output_padding=0,groups=1, bias=True, dilation=1, padding_mode='zeros')

        self.gen = nn.Sequential(
            Generator._block(z_dim, gf_dim*4, padding=0),
            Generator._block(gf_dim*4, gf_dim*2),
            Generator._block(gf_dim*2, gf_dim*1),
            Generator._block(gf_dim*1, c_dim, final_layer=True),
        )

    def forward(self,inp):
        return self.gen(
            inp
        ) # (c_dim , 215 , 215 )

    @staticmethod
    def _block(in_channels, out_channels, kernel_size=5, stride=3, padding=1, final_layer=False):
        if final_layer:
            return nn.Sequential(
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
                nn.Tanh()
            )
        else:
            return nn.Sequential(
                nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ELU(inplace = True)
            )

In [ ]:
# TEST
gen = Generator(100, 1, 10)
gen(get_noise(1, 100)).shape

In [5]:
from collections import OrderedDict
import torch
import torch.nn as nn


class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features
        self.encoder1 = UNet._block(in_channels, features, name="enc1")
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder2 = UNet._block(features, features * 2, name="enc2")
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder3 = UNet._block(features * 2, features * 4, name="enc3")
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.encoder4 = UNet._block(features * 4, features * 8, name="enc4")
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)

        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")

        self.upconv4 = nn.ConvTranspose2d(
            features * 16, features * 8, kernel_size=2, stride=2
        )
        self.decoder4 = UNet._block((features * 8) * 2, features * 8, name="dec4")
        self.upconv3 = nn.ConvTranspose2d(
            features * 8, features * 4, kernel_size=2, stride=2
        )
        self.decoder3 = UNet._block((features * 4) * 2, features * 4, name="dec3")
        self.upconv2 = nn.ConvTranspose2d(
            features * 4, features * 2, kernel_size=2, stride=2
        )
        self.decoder2 = UNet._block((features * 2) * 2, features * 2, name="dec2")
        self.upconv1 = nn.ConvTranspose2d(
            features * 2, features, kernel_size=2, stride=2
        )
        self.decoder1 = UNet._block(features * 2, features, name="dec1")

        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )

    def forward(self, x):
        self.z1 = self.encoder1(x)
        self.z2 = self.encoder2(self.pool1(self.z1))
        self.z3 = self.encoder3(self.pool2(self.z2))
        self.z4 = self.encoder4(self.pool3(self.z3))

        self.z5 = self.bottleneck(self.pool4(self.z4))

        dec4 = self.upconv4(self.z5)
        dec4 = torch.cat((dec4, self.z4), dim=1)

        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, self.enc3), dim=1)

        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, self.enc2), dim=1)

        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, self.enc1), dim=1)

        dec1 = self.decoder1(dec1)
        return torch.sigmoid(self.conv(dec1))

    @staticmethod
    def _block(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                ]
            )
        )

In [ ]:
from collections import OrderedDict

import torch
import torch.nn as nn


class UNet(nn.Module):

    def __init__(self, in_channels=3, out_channels=1, init_features=32):
        super(UNet, self).__init__()

        features = init_features

        # Encoder
        self.encoder1 = UNet._encoder_layer(in_channels , features * 1, name="enc1")
        self.encoder2 = UNet._encoder_layer(features * 1, features * 2, name="enc2")
        self.encoder3 = UNet._encoder_layer(features * 2, features * 4, name="enc3")
        self.encoder4 = UNet._encoder_layer(features * 4, features * 8, name="enc4")

        # important
        self.bottleneck = UNet._block(features * 8, features * 16, name="bottleneck")

        # Decoder
        self.decoder4 = UNet._decoder_layer((features * 16) * 2, features * 8, name="dec4")
        self.decoder3 = UNet._decoder_layer((features *  8) * 2, features * 4, name="dec3")
        self.decoder2 = UNet._decoder_layer((features *  4) * 2, features * 2, name="dec2")
        self.decoder1 = UNet._decoder_layer((features *  2) * 2, features * 1, name="dec1")


        self.conv = nn.Conv2d(
            in_channels=features, out_channels=out_channels, kernel_size=1
        )

    def forward(self, x):
        self.enc1 = self.encoder1(x)
        self.enc2 = self.encoder2(self.enc1)
        self.enc3 = self.encoder3(self.enc2)
        self.enc4 = self.encoder4(self.enc3)

        self.z5 = self.bottleneck(self.pool4(self.z4))

        dec4 = self.upconv4(self.z5)
        dec4 = torch.cat((dec4, self.z4), dim=1)

        dec4 = self.decoder4(dec4)
        dec3 = self.upconv3(dec4)
        dec3 = torch.cat((dec3, self.z3), dim=1)

        dec3 = self.decoder3(dec3)
        dec2 = self.upconv2(dec3)
        dec2 = torch.cat((dec2, self.z2), dim=1)

        dec2 = self.decoder2(dec2)
        dec1 = self.upconv1(dec2)
        dec1 = torch.cat((dec1, self.z1), dim=1)

        dec1 = self.decoder1(dec1)
        return torch.sigmoid(self.conv(dec1))

    @staticmethod
    def _encoder_layer(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=in_channels,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True)),
                    (
                        name + 'maxpool', nn.MaxPool2d(kernel_size=2, stride=2)
                    )
                ]
            )
        )

    @staticmethod
    def _decoder_layer(in_channels, features, name):
        return nn.Sequential(
            OrderedDict(
                [
                    (
                        name + "upconv", nn.ConvTranspose2d(in_channels, features, kernel_size=2, stride=2)
                    ),
                    (
                        name + "conv1",
                        nn.Conv2d(
                            in_channels=2*features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm1", nn.BatchNorm2d(num_features=features)),
                    (name + "relu1", nn.ReLU(inplace=True)),
                    (
                        name + "conv2",
                        nn.Conv2d(
                            in_channels=features,
                            out_channels=features,
                            kernel_size=3,
                            padding=1,
                            bias=False,
                        ),
                    ),
                    (name + "norm2", nn.BatchNorm2d(num_features=features)),
                    (name + "relu2", nn.ReLU(inplace=True))
                ]
            )
        )

In [17]:
m = nn.Sequential(
    OrderedDict([
        ('fc1', nn.Linear(10,20)),
        ('fc2', nn.Linear(20,30)),
    ])
)
m.fc1

Linear(in_features=10, out_features=20, bias=True)

In [ ]:
# Change activation to RELU
class Critic(nn.Module):
    def __init__(self , c_dim , df_dim):
        super(Critic, self).__init__()

        # torch.nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride=1, padding=0,
        #                          output_padding=0,groups=1, bias=True, dilation=1, padding_mode='zeros')

        self.critic = nn.Sequential(
            Critic._block(c_dim, df_dim),
            Critic._block(df_dim, df_dim*2),
            Critic._block(df_dim*2, df_dim*4),
            Critic._block(df_dim*4, 1, padding=0, final_layer=True),
        )

    def forward(self,inp):
        return self.critic(
            inp
        ).flatten(1) # (N , 1 , 1, 1 )

    @staticmethod
    def _block(in_channels, out_channels, kernel_size=5, stride=3, padding=1, final_layer=False):
        if final_layer:
            return nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        else:
            return nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ELU(inplace = True)
            )

In [ ]:
# TEST
N, C, H, W = 10, 1, 135, 135
x = torch.ones(N, C, H, W)
dis = Critic(C, 10)
dis(x).shape

### Loss

In [ ]:
def get_epsilon(N_epsilon, device='cpu'):
    """
    Parameters:
        N_epsilon : Number of epsilons
    Output:
        e.g. shape : (128, 1, 1, 1)
    """
    return torch.rand(N_epsilon, 1, 1, 1, device=device, requires_grad=True)

In [ ]:
def get_gradient(crit, data):
    """
    Return the gradient of the critic's scores with respect to mixes of real and fake images.
    Parameters:
        dis: the critic model
        data: a batch of data
    Returns:
        gradient: the gradient of the discriminator's scores, with respect to data
    """

    # True require_grad of Data
    data.requires_grad_()
    # Calculate the discriminator's scores on the data
    score = crit(data)

    # Take the gradient of the scores with respect to the data
    gradient = torch.autograd.grad(
        inputs=data,
        outputs=score,
        # These other parameters have to do with the pytorch autograd engine works
        grad_outputs=torch.ones_like(score),
        create_graph=True,
        retain_graph=True,
    )[0]
    return gradient

In [ ]:
def gradient_penalty(gradient):
    '''
    Return the gradient penalty, given a gradient.
    Given a batch of data gradients, you calculate the magnitude of each data's gradient
    and penalize the mean quadratic distance of each magnitude to 1.

    Parameters:
        gradient: the gradient of the discriminator's scores, with respect to the data
        e.g shape  : (128, 1, 28, 28)

    Returns:
        penalty: the gradient penalty
        e.g shaoe : (scaler)
    '''

    # Flatten the gradients so that each row captures one image
    # e.g shape  : (128, 1, 28, 28) ==> (128, 784)
    gradient = gradient.view(len(gradient), -1)

    # Calculate the magnitude of every row
    # e.g shape : (128, 784) ==> (128, 1)
    gradient_norm = gradient.norm(2, dim=1)

    # Penalize the mean squared distance of the gradient norms from 1
    # e.g shape : (128, 1) ==> (scaler)
    penalty = ( ( gradient_norm - 1.0 )**2 ).mean(dim=0)
    return penalty

In [ ]:
def get_crit_loss(
    image, z_dim,
    gen, crit,
    c_lambda,
    device):
    '''
    Parameters:
        All as same as DCGAN Losses
    Returns:
        dis_loss: a scalar for the dis's loss
    '''
    #x_no_tumor_fake = gen(x_tumor #+ get_noise(x_tumor, device=device)).detach()

    image_hat = gen(
        get_noise(image.shape[0], z_dim, device=device)
    ).detach()

    # Mixed "real" with "fake"
    #epsilon = get_epsilon(N_epsilon = x_no_tumor.shape[0], device=device)
    epsilon = get_epsilon(N_epsilon = image.shape[0], device=device)

    #mixed_images = x_no_tumor * epsilon + x_no_tumor_fake * (1 - epsilon)
    mixed_images = image * epsilon + image_hat * (1 - epsilon)

    # Calculate Gradient Penalty (use prior funcs)
    gp = gradient_penalty(get_gradient(crit, mixed_images))
    # Calculate "Line 7" of "Algorithm 1" in main paper
    #return ( dis(x_no_tumor_fake) - dis(x_no_tumor) + c_lambda * gp ).mean(dim=0)
    return ( crit(image_hat) - crit(image) + c_lambda * gp ).mean(dim=0)

In [ ]:
def get_gen_loss(
      image,z_dim,
      gen, crit,
      device):
    '''
    Return the loss of a generator.
    Parameters:

    Returns:
       a scalar loss value for the current batch of the generator
    '''
    image_hat = gen(
        get_noise(image.shape[0], z_dim, device=device)
    )
    # Calculate "Line 12" of "Algorithm 1" in main paper
    return ( -crit(image_hat) ).mean(dim=0)

### Helper

In [ ]:
def ploter(image, image_hat):
    """
    (H, W)
    """
    plt.figure()
    plt.subplot(1,2,1)
    #plt.imshow(image_hat, cmap='gray', vmin=-1, vmax=1)
    plt.imshow(image_hat)
    plt.tight_layout()
    plt.title("Reconstruct")

    plt.subplot(1,2,2)
    #plt.imshow(image, cmap='gray', vmin=-1, vmax=1)
    plt.imshow(image)
    plt.tight_layout()
    plt.title("Original")

    plt.show()

In [ ]:
def weights_init(submodules):
    if isinstance(submodules, nn.Conv2d) or isinstance(submodules, nn.ConvTranspose2d):
        torch.nn.init.normal_(submodules.weight, 0.0, 0.02)
    if isinstance(submodules, nn.BatchNorm2d):
        torch.nn.init.normal_(submodules.weight, 0.0, 0.02)
        torch.nn.init.constant_(submodules.bias, 0)

In [ ]:
def show_tensor_images(image_tensor, num_images=25, size=(1, 128, 128)):
    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze(), cmap='gray', vmin=-1, vmax=1)
    plt.show()

### Hyperparamters

In [ ]:
z_dim , c_dim , gf_dim = 100, 1, 64
df_dim = 64

device = 'cuda'

batch_size =20


lr_gen, lr_crit = 0.002, 0.002
beta_1 = 0.5
beta_2 = 0.999

# epochs
epochs= 300
disp_freq=1
display_step=5

crit_repeats=5
c_lambda = 10

In [ ]:
gen = Generator(z_dim , c_dim , gf_dim).to(device)
crit = Critic(c_dim, df_dim).to(device)

# Initialize
gen = gen.apply(weights_init)
crit = crit.apply(weights_init)

# Loss function
#criterion=nn.BCEWithLogitsLoss()

# Optimizers
optim_crit = torch.optim.Adam(crit.parameters(), lr=lr_crit, betas=(beta_1, beta_2))
optim_gen = torch.optim.Adam(gen.parameters(), lr=lr_gen, betas=(beta_1, beta_2))

### Dataset

In [ ]:
root = "./../dataset/kaggle1/no"
dataset = Anomaly_Dataset(root)
dataloader_normal = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for x, y in dataloader_normal:
    x=x
    y=y

In [ ]:
x.shape

In [ ]:
get_noise(100,100).dtype

In [ ]:
plt.figure()
plt.imshow(x[0], cmap='gray', vmin=-1, vmax=1)

In [ ]:
import gc

gc.collect()

torch.cuda.empty_cache()

### Training

In [ ]:
cur_step = 0
generator_losses = []
critic_losses = []

for epoch in range(1,epochs+1):
    print(60 * "#")
    print(6 * "#" + " Epoch " + str(epoch) + " " + 45 * "#")
    print(60 * "#")

    # Set mode on "train mode"
    gen.train()
    dis.train()

    for image,_ in dataloader_normal:
        cur_batch_size = len(image)
        # GPU (model and data)
        image = image.unsqueeze(1).to(device)

        # Discriminator Learning
        mean_iteration_critic_loss = 0
        for _ in range(crit_repeats):
          optim_crit.zero_grad()
          crit_loss  = get_crit_loss(image, z_dim, gen, crit, c_lambda, device=device)
          crit_loss .backward(retain_graph=True)
          optim_crit.step()
          mean_iteration_critic_loss += crit_loss.item() / crit_repeats
        critic_losses += [mean_iteration_critic_loss]

        # Generator Learning
        optim_gen.zero_grad()
        gen_loss  = get_gen_loss(image, z_dim, gen, crit, device=device)
        gen_loss .backward()
        optim_gen.step()
        generator_losses += [gen_loss .item()]

    # Save parameters of discriminator and generator
    #save_model(gen, dis, epoch, root_models, mode='colab')
    #print("Loss Dis: {:.2f}\tLoss Gen: {:.2f}".format(loss_dis,loss_gen))

    ### Visualization code ###
    if cur_step % display_step == 0 and cur_step > 0:
        gen_mean = sum(generator_losses[-display_step:]) / display_step
        crit_mean = sum(critic_losses[-display_step:]) / display_step
        print(f"Epoch {epoch}, step {cur_step}: Generator loss: {gen_mean}, critic loss: {crit_mean}")



        fake = gen(
            get_noise(image.shape[0], z_dim, device)
        )

        #show_tensor_images(fake)
        #show_tensor_images(image)

        i=math.floor(np.random.uniform(0, image.shape[0]))
        plt.figure()
        ploter(image[i,0].detach().cpu(), fake[i,0].detach().cpu())
        plt.show()

        step_bins = 20
        num_examples = (len(generator_losses) // step_bins) * step_bins
        plt.plot(
            range(num_examples // step_bins),
            torch.Tensor(generator_losses[:num_examples]).view(-1, step_bins).mean(1),
            label="Generator Loss"
        )
        plt.plot(
            range(num_examples // step_bins),
            torch.Tensor(critic_losses[:num_examples]).view(-1, step_bins).mean(1),
            label="Critic Loss"
        )
        plt.legend()
        plt.show()
    cur_step += 1

In [ ]:
image.shape

### Dataset

In [ ]:
root = "./../dataset/kaggle1/no"
dataset = Anomaly_Dataset(root)
dataloader_normal = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [ ]:
for x, y in dataloader_normal:
    x=x
    y=y

In [ ]:
plt.figure()
plt.imshow(x[0], cmap='gray', vmin=-1, vmax=1)

### Training